In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [43]:
md_lk = pd.read_csv('Dataset/links.csv')
md_lk.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [49]:
md_lk.isnull().sum()

movieId    0
imdbId     0
tmdbId     0
dtype: int64

In [48]:
md_lk.dropna(how='any',inplace=True)

In [56]:
md_lk.tmdbId = md_lk['tmdbId'].astype(int)

In [80]:
md_rt = pd.read_csv('Dataset/ratings_small.csv')
md_rt.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [81]:
md_rt.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [82]:
df_rt = pd.merge(md_rt, md_lk, on='movieId')

In [83]:
df_rt.drop(['timestamp', 'imdbId','movieId'], axis=1, inplace=True)
df_rt.head(5)

,userId,rating,tmdbId
0,1,2.5,9909
1,7,3.0,9909
2,31,4.0,9909
3,32,4.0,9909
4,36,3.0,9909


In [89]:
reader = Reader()

In [90]:
data = Dataset.load_from_df(df_rt[['userId', 'tmdbId', 'rating']], reader)
data.split(n_folds=5)

In [92]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8973
MAE:  0.6907
------------
Fold 2
RMSE: 0.8906
MAE:  0.6875
------------
Fold 3
RMSE: 0.8994
MAE:  0.6942
------------
Fold 4
RMSE: 0.9000
MAE:  0.6910
------------
Fold 5
RMSE: 0.8917
MAE:  0.6867
------------
------------
Mean RMSE: 0.8958
Mean MAE : 0.6900
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8972793881011317,
                             0.8905898349207046,
                             0.8994020698030583,
                             0.9000396144040528,
                             0.8916610681808937],
                            'mae': [0.6907254871580312,
                             0.6875371542620315,
                             0.6941861978855383,
                             0.6909947750200793,
                             0.6867246267272353]})

In [93]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [94]:
df_rt[df_rt['userId'] == 1]

,userId,rating,tmdbId
0,1,2.5,9909
42,1,3.0,11360
84,1,3.0,819
117,1,2.0,1103
165,1,4.0,11216
211,1,2.0,11778
259,1,2.0,665
305,1,2.0,783
351,1,3.5,6114
403,1,2.0,1598


In [95]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.6010140638161223, details={'was_impossible': False})